In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datatset/train.csv
/kaggle/input/datatset/test.csv
/kaggle/input/datatset/gender_submission.csv


In [2]:
test = pd.read_csv("/kaggle/input/datatset/test.csv")
data = pd.read_csv("/kaggle/input/datatset/train.csv")
test_ids = test["PassengerId"]


In [3]:
def clean(data):
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis = 1)
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)
    data.Embarked.fillna("U", inplace=True)
    return data
data = clean(data)
test = clean(test)

/tmp/ipykernel_17/49802552.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(data[col].median(), inplace=True)
/tmp/ipykernel_17/49802552.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
cols = ["Sex", "Embarked"]
for col in cols:
    data[col] = le.fit_transform(data[col])
    test[col] = le.fit_transform(test[col])
    print(le.classes_)


['female' 'male']
['C' 'Q' 'S']


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
y = data["Survived"]
x = data.drop(["Survived"], axis = 1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state = 42)

In [6]:
clf = LogisticRegression(random_state = 0, max_iter=1000).fit(x_train, y_train)


In [7]:
prediction = clf.predict(x_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, prediction)

0.8100558659217877

In [8]:
submission_preds = clf.predict(test)

In [9]:
df = pd.DataFrame({"PassengerId": test_ids.values, "Survived": submission_preds})

In [10]:
df.to_csv("Titanic.csv", index=False)